In [1]:
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None

from neo4j import GraphDatabase

# Connect to Neo4j
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "h4ck3r20o8"))

In [2]:
pokemon = pd.read_csv("../data/pokemonDB_dataset.csv")

In [3]:
def clear_neo4j_database(driver):
    with driver.session() as session:
        session.run("MATCH (n) DETACH DELETE n")

# Clear the Neo4j database
clear_neo4j_database(driver)

In [4]:
pokemon.head()

,Pokemon,Type,Species,Height,Weight,Abilities,EV Yield,Catch Rate,Base Friendship,Base Exp,Growth Rate,Egg Groups,Gender,Egg Cycles,HP Base,HP Min,HP Max,Attack Base,Attack Min,Attack Max,Defense Base,Defense Min,Defense Max,Special Attack Base,Special Attack Min,Special Attack Max,Special Defense Base,Special Defense Min,Special Defense Max,Speed Base,Speed Min,Speed Max
0,Abomasnow,"Grass, Ice",Frost Tree Pokémon,2.2 m (7′03″),135.5 kg (298.7 lbs),"1. Snow Warning, Soundproof (hidden ability)","1 Attack, 1 Sp. Atk","60 (7.8% with PokéBall, full HP)",50 (normal),173,Slow,"Grass, Monster","50% male, 50% female","20 (4,884–5,140 steps)",90,290,384,92,170,311,75,139,273,92,170,311,85,157,295,60,112,240
1,Mega Abomasnow,"Grass, Ice",Frost Tree Pokémon,2.7 m (8′10″),185.0 kg (407.9 lbs),1. Snow Warning,"1 Attack, 1 Sp. Atk","60 (7.8% with PokéBall, full HP)",50 (normal),208,Slow,"Grass, Monster","50% male, 50% female","20 (4,884–5,140 steps)",90,290,384,132,242,399,105,193,339,132,242,399,105,193,339,30,58,174
2,Abra,Psychic,Psi Pokémon,0.9 m (2′11″),19.5 kg (43.0 lbs),"1. Synchronize, 2. Inner Focus, Magic Guard (hidden ability)",1 Sp. Atk,"200 (26.1% with PokéBall, full HP)",50 (normal),62,Medium Slow,Human-Like,"75% male, 25% female","20 (4,884–5,140 steps)",25,160,254,20,40,152,15,31,141,105,193,339,55,103,229,90,166,306
3,Absol,Dark,Disaster Pokémon,1.2 m (3′11″),47.0 kg (103.6 lbs),"1. Pressure, 2. Super Luck, Justified (hidden ability)",2 Attack,"30 (3.9% with PokéBall, full HP)",35 (lower than normal),163,Medium Slow,Field,"50% male, 50% female","25 (6,169–6,425 steps)",65,240,334,130,238,394,60,112,240,75,139,273,60,112,240,75,139,273
4,Mega Absol,Dark,Disaster Pokémon,1.2 m (3′11″),49.0 kg (108.0 lbs),1. Magic Bounce,2 Attack,"30 (3.9% with PokéBall, full HP)",35 (lower than normal),198,Medium Slow,Field,"50% male, 50% female","25 (6,169–6,425 steps)",65,240,334,150,274,438,60,112,240,115,211,361,60,112,240,115,211,361


### Pokemon

In [10]:
pokemon.to_parquet("../data/pokemonDB_dataset.parquet", index=False)

In [12]:
def insert_pokemon_data(tx, row):
    query = """
    MERGE (p:Pokemon {name: $pokemon})
    SET p.height = $height,
        p.weight = $weight,
        p.ev_yield = $ev_yield,
        p.catch_rate = $catch_rate,
        p.base_friendship = $base_friendship,
        p.base_exp = $base_exp,
        p.growth_rate = $growth_rate,
        p.egg_cycles = $egg_cycles,
        p.hp_base = $hp_base,
        p.hp_min = $hp_min,
        p.hp_max = $hp_max,
        p.attack_base = $attack_base,
        p.attack_min = $attack_min,
        p.attack_max = $attack_max,
        p.defense_base = $defense_base,
        p.defense_min = $defense_min,
        p.defense_max = $defense_max,
        p.special_attack_base = $special_attack_base,
        p.special_attack_min = $special_attack_min,
        p.special_attack_max = $special_attack_max,
        p.special_defense_base = $special_defense_base,
        p.special_defense_min = $special_defense_min,
        p.special_defense_max = $special_defense_max,
        p.speed_min = $speed_min,
        p.speed_max = $speed_max
        p.description = $description
    """
    tx.run(query, 
           pokemon=row['Pokemon'], 
           height=row['Height'], 
           weight=row['Weight'], 
           ev_yield=row['EV Yield'], 
           catch_rate=row['Catch Rate'], 
           base_friendship=row['Base Friendship'], 
           base_exp=row['Base Exp'], 
           growth_rate=row['Growth Rate'], 
           egg_cycles=row['Egg Cycles'], 
           hp_base=row['HP Base'], 
           hp_min=row['HP Min'], 
           hp_max=row['HP Max'], 
           attack_base=row['Attack Base'], 
           attack_min=row['Attack Min'], 
           attack_max=row['Attack Max'], 
           defense_base=row['Defense Base'], 
           defense_min=row['Defense Min'], 
           defense_max=row['Defense Max'], 

           special_attack_base=row['Special Attack Base'], 
           special_attack_min=row['Special Attack Min'],
           special_attack_max=row['Special Attack Max'],
           special_defense_base=row['Special Defense Base'],
           special_defense_min=row['Special Defense Min'],
           special_defense_max=row['Special Defense Max'],

           speed_min=row['Speed Min'], 
           speed_max=row['Speed Max'],
           description=row['description']
           )

# Insert data into Neo4j
with driver.session() as session:
    for index, row in pokemon.iterrows():
        session.execute_write(insert_pokemon_data, row)

KeyError: 'description'

In [7]:
pokemon.head()

,Pokemon,Type,Species,Height,Weight,Abilities,EV Yield,Catch Rate,Base Friendship,Base Exp,Growth Rate,Egg Groups,Gender,Egg Cycles,HP Base,HP Min,HP Max,Attack Base,Attack Min,Attack Max,Defense Base,Defense Min,Defense Max,Special Attack Base,Special Attack Min,Special Attack Max,Special Defense Base,Special Defense Min,Special Defense Max,Speed Base,Speed Min,Speed Max,Description
0,Abomasnow,"Grass, Ice",Frost Tree Pokémon,2.2 m (7′03″),135.5 kg (298.7 lbs),"1. Snow Warning, Soundproof (hidden ability)","1 Attack, 1 Sp. Atk","60 (7.8% with PokéBall, full HP)",50 (normal),173,Slow,"Grass, Monster","50% male, 50% female","20 (4,884–5,140 steps)",90,290,384,92,170,311,75,139,273,92,170,311,85,157,295,60,112,240,Abomasnow!
1,Mega Abomasnow,"Grass, Ice",Frost Tree Pokémon,2.7 m (8′10″),185.0 kg (407.9 lbs),1. Snow Warning,"1 Attack, 1 Sp. Atk","60 (7.8% with PokéBall, full HP)",50 (normal),208,Slow,"Grass, Monster","50% male, 50% female","20 (4,884–5,140 steps)",90,290,384,132,242,399,105,193,339,132,242,399,105,193,339,30,58,174,"Mega Abomasnow is a powerful, Snow/Ice-type Pokémon that was initially unavailable in the mainline games. In Pokémon X and Y, it was revealed that Abomasnow, a common snowman-like Pokémon, could mega evolve into Mega Abomasnow using a Mega Stone. This transformation gives Mega Abomasnow an incredible boost in power, including increased speed, special attack prowess, and resilience.\n\nMega Abomasnow's primary characteristics include its extraordinary speed, which enables it to dodge attacks with ease, and its incredibly potent Blizzard attack, capable of dealing significant damage to opponents. Additionally, its high defense stat ensures that it can withstand heavy hits before retaliating.\n\nOverall, Mega Abomasnow is a formidable opponent on the battlefield, offering a unique combination of defensive and offensive capabilities that make it a valuable ally in any Pokémon trainer's team."
2,Abra,Psychic,Psi Pokémon,0.9 m (2′11″),19.5 kg (43.0 lbs),"1. Synchronize, 2. Inner Focus, Magic Guard (hidden ability)",1 Sp. Atk,"200 (26.1% with PokéBall, full HP)",50 (normal),62,Medium Slow,Human-Like,"75% male, 25% female","20 (4,884–5,140 steps)",25,160,254,20,40,152,15,31,141,105,193,339,55,103,229,90,166,306,"Abra is the eighth-generation Pokémon introduced in the 2020 Nintendo Switch game, Pokémon Legends: Arceus. It is an Ice/Psychic-type Pokémon that is known for its unique ability to use its psychic powers to levitate and move objects without physically touching them. Abra has large eyes and a slender, rabbit-like body with a long ear on the top of its head."
3,Absol,Dark,Disaster Pokémon,1.2 m (3′11″),47.0 kg (103.6 lbs),"1. Pressure, 2. Super Luck, Justified (hidden ability)",2 Attack,"30 (3.9% with PokéBall, full HP)",35 (lower than normal),163,Medium Slow,Field,"50% male, 50% female","25 (6,169–6,425 steps)",65,240,334,130,238,394,60,112,240,75,139,273,60,112,240,75,139,273,"Here is a brief summary about Absol:\n\nAbsol is a Pokémon species that first appears in the third generation of the Pokémon franchise. It is classified as a Dark/Psychic-type Pokémon. Absol has four sharp horns on its head, which are used to sense and analyze its surroundings. Its eyes glow red when it detects danger or abnormal activity nearby. It is known for its exceptional instincts and ability to predict natural disasters like earthquakes and storms. In the wild, Absol tends to live in areas prone to such calamities, where it can utilize its keen senses to anticipate and prepare for them."
4,Mega Absol,Dark,Disaster Pokémon,1.2 m (3′11″),49.0 kg (108.0 lbs),1. Magic Bounce,2 Attack,"30 (3.9% with PokéBall, full HP)",35 (lower than normal),198,Medium Slow,Field,"50% male, 50% female","25 (6,169–6,425 steps)",65,240,334,150,274,438,60,112,240,115,211,361,60,112,240,115,211,361,"Mega Absol is a fictional Pokémon species that was introduced in 2013 as part of the Pokémon franchise. It is a Rock/Poison-type Pokémon and is cons


### Pokemon -> Egg Group


In [7]:
pokemon_egg_groups = pokemon['Egg Groups'].str.split(', ', expand=True).stack().reset_index(level=1, drop=True)
pokemon_expanded_egg_groups = pokemon.drop('Egg Groups', axis=1).join(pokemon_egg_groups.rename('Egg Group'))

def insert_egg_group_data(tx, pokemon, egg_group):
    query = """
    MATCH (p:Pokemon {name: $pokemon})
    MERGE (e:EggGroup {name: $egg_group})
    MERGE (p)-[:HAS_GENETIC_TIE]->(e)
    """
    tx.run(query, pokemon=pokemon, egg_group=egg_group)

# Insert egg group data into Neo4j
with driver.session() as session:
    for index, row in pokemon_expanded_egg_groups.iterrows():
        session.execute_write(insert_egg_group_data, row['Pokemon'], row['Egg Group'])

### Pokemon -> Gender

In [8]:
pokemon_genders = pokemon['Gender'].str.split(', ', expand=True).stack().reset_index(level=1, drop=True)
pokemon_expanded_genders = pokemon.drop('Gender', axis=1).join(pokemon_genders.rename('Gender'))

def insert_gender_data(tx, pokemon, gender):
    query = """
    MATCH (p:Pokemon {name: $pokemon})
    MERGE (g:Gender {name: $gender})
    MERGE (p)-[:HAS_GENDER]->(g)
    """
    tx.run(query, pokemon=pokemon, gender=gender)

# Insert gender data into Neo4j
with driver.session() as session:
    for index, row in pokemon_expanded_genders.iterrows():
        session.execute_write(insert_gender_data, row['Pokemon'], row['Gender'])


### Pokemon -> Abilities

In [9]:
pokemon_abilities = pokemon['Abilities'].str.split(', ', expand=True).stack().reset_index(level=1, drop=True)
pokemon_abilities = pokemon_abilities.str.replace(r'^\d+\.\s*', '', regex=True)
pokemon_expanded_abilities = pokemon.drop('Abilities', axis=1).join(pokemon_abilities.rename('Ability'))

def insert_ability_data(tx, pokemon, ability):
    query = """
    MATCH (p:Pokemon {name: $pokemon})
    MERGE (a:Ability {name: $ability})
    MERGE (p)-[:HAS_ABILITY]->(a)
    """
    tx.run(query, pokemon=pokemon, ability=ability)

# Insert ability data into Neo4j
with driver.session() as session:
    for index, row in pokemon_expanded_abilities.filter(['Pokemon', 'Ability']).iterrows():
        session.execute_write(insert_ability_data, row['Pokemon'], row['Ability'])

### Pokemon -> Species

In [10]:
pokemon['Species'] = pokemon['Species'].str.replace(' Pokémon', '')
species_df = pokemon.filter(['Species']).drop_duplicates()

# Define a function to insert species data into Neo4j
def insert_species_data(tx, species):
    query = """
    MERGE (s:Species {name: $species})
    """
    tx.run(query, species=species)

# Insert species data into Neo4j
with driver.session() as session:
    for index, row in species_df.iterrows():
        session.execute_write(insert_species_data, row['Species'])

### Species -> Type

In [11]:
pokemon_types = pokemon['Type'].str.split(', ', expand=True).stack().reset_index(level=1, drop=True)
pokemon_expanded = pokemon.drop('Type', axis=1).join(pokemon_types.rename('Type'))

def insert_species_type_data(tx, species, pokemon):
    query = """
    MATCH (s:Species {name: $species})
    MERGE (p:Pokemon {name: $pokemon})
    MERGE (p)-[:BELONGS_TO]->(s)
    """
    tx.run(query, species=species, pokemon=pokemon)

def insert_pokemon_type_data(tx, pokemon, type_):
    query = """
    MATCH (p:Pokemon {name: $pokemon})
    MERGE (t:Type {name: $type_})
    MERGE (p)-[:HAS_TYPE]->(t)
    """
    tx.run(query, pokemon=pokemon, type_=type_)

# Insert species, type, and pokemon-type data into Neo4j
with driver.session() as session:
    for index, row in pokemon_expanded.filter(['Species', 'Type', 'Pokemon']).iterrows():
        session.execute_write(insert_species_type_data, row['Species'], row['Pokemon'])
        session.execute_write(insert_pokemon_type_data, row['Pokemon'], row['Type'])


In [ ]:
# Query the type and species of Pikachu from Neo4j
def query_pikachu_type_and_species(tx):
    query = """
    MATCH (p:Pokemon {name: 'Pikachu'})-[:HAS_TYPE]->(t:Type),
          (p)-[:BELONGS_TO]->(s:Species)
    RETURN t.name AS Type, s.name AS Species
    """
    result = tx.run(query)
    return result.single()

# Execute the query and print the result
with driver.session() as session:
    pikachu_data = session.execute_read(query_pikachu_type_and_species)
    if pikachu_data:
        print(f"Pikachu Type: {pikachu_data['Type']}, Species: {pikachu_data['Species']}")
    else:
        print("Pikachu data not found.")

In [ ]:
pokemon['Pokemon'] = pokemon['Pokemon'].str.replace(' ', '_')
pokemon.head()

### IMAGE EMBEDDINGS

In [ ]:
import torch
import torchvision.models as models

# Load pretrained ResNet152 model
model = models.resnet152(weights=True)

# Remove the final fully-connected layer
model = torch.nn.Sequential(*(list(model.children())[:-1]))

# Set the model to evaluation mode
model.eval()

In [46]:
from torchvision import transforms
from PIL import Image
from urllib.parse import quote

# Define preprocessing steps
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])

# Define a function to extract embeddings for each Pokemon
def extract_embeddings_for_pokemon(pokemon_name):

    encoded_name = quote(pokemon_name)
    # INSERT_YOUR_CODE
    try:
        image_path = f"../data/pokemon_db/images/{pokemon_name}/{pokemon_name}_new.png"
        # Load and preprocess the image
        image = Image.open(image_path).convert("RGB")
        input_tensor = preprocess(image).unsqueeze(0)
        
        # Extract embeddings
        with torch.no_grad():
            embeddings = model(input_tensor)
        
        # Reshape embeddings to a 1D vector
        return embeddings.squeeze().numpy()
    except FileNotFoundError:
        print(f"Image not found for {pokemon_name}. Continuing to next.")
        return None

In [ ]:

def insert_embeddings(tx, pokemon_name, embeddings):
    # Create a Cypher query to insert embeddings
    query = """
    MERGE (p:Pokemon {name: $pokemon_name})
    SET p.resnet152_embeddings = $embeddings
    """
    # Execute the query with the provided parameters
    tx.run(query, pokemon_name=pokemon_name, embeddings=embeddings.tolist())

for index, row in pokemon.iterrows():
    pokemon_name = row['Pokemon']

    embeddings = extract_embeddings_for_pokemon(pokemon_name)
    
    if embeddings is not None:
        with driver.session() as session:
            session.execute_write(insert_embeddings, pokemon_name, embeddings)

In [92]:
# INSERT_YOUR_CODE
def query_similar_pokemon(tx, target_embeddings, threshold=0.7):
    # Create a Cypher query to find similar Pokemon based on embeddings
    query = """
    MATCH (p:Pokemon)
    WHERE p.resnet152_embeddings IS NOT NULL AND size(p.resnet152_embeddings) > 0
    WITH p, gds.similarity.cosine(p.resnet152_embeddings, $target_embeddings) as cosineSimilarity
    WHERE cosineSimilarity > $threshold
    RETURN p.name AS name, cosineSimilarity
    ORDER BY cosineSimilarity DESC
    """
    # Execute the query with the provided parameters
    result = tx.run(query, target_embeddings=target_embeddings.tolist(), threshold=threshold)
    return [(record["name"], record["cosineSimilarity"]) for record in result]

In [ ]:
embeddings = extract_embeddings_for_pokemon("Pikachu")
embeddings

In [ ]:
# Use the driver to open a session and query for similar Pokemon
with driver.session() as session:
    similar_pokemon = session.execute_write(query_similar_pokemon, embeddings, 0.8)

# Print the similar Pokemon}
for name, cosineSimilarity in similar_pokemon:
    print(f"Pokemon: {name}, Similarity: {cosineSimilarity}")